# Compare bipolar and unipolar electrodes montages (MC)

In this notebook, we compare the effect of bipolar and unipolar electrodes montages targetting the motor cortex (MC).

In [ ]:
%load_ext lab_black

In [ ]:
# Setup
from pathlib import Path
import os
import sys

import arviz as az
import bambi as bmb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add utility package to path
sys.path.append(os.environ.get("BRAINWEB_TDCS_CODE_DIR", "../code"))
from brainweb_tdcs import ROIS
from brainweb_tdcs.study import get_experiments_for_roi
from brainweb_tdcs.plot import (
    display_side_by_side,
    plot_bipolar_unipolar,
    plot_posterior,
)

# Set path data directory
if "BRAINWEB_TDCS_DATA_DIR" not in os.environ:
    os.environ["BRAINWEB_TDCS_DATA_DIR"] = str((Path.cwd() / "../data").resolve())

# Set the random seed
RANDOM_SEED = 1234

In [ ]:
CAPTIONS = {
    "e": "Magnitude of the\nelectric field",
    "e_r": "Magnitude of the\nnormal component of electric field",
}
LABELS = {
    "e": "$| \mathbf{e} |$ (mVm$^{-1}$)",
    "e_r": "$| \mathbf{e}_r |$ (mVm$^{-1}$)",
}
FUNC_DICT = {
    "mean": np.mean,
    "std": np.std,
    "2.5%": lambda x: np.percentile(x, 2.5),
    "97.5%": lambda x: np.percentile(x, 97.5),
}

## Define the experiments

We have considered two different electrodes montages for the MC. The first used the C3-C4 bipolar electrodes montage while the second used the C3-Fp2 unipolar electrodes montage.

We first select these experiments:

In [ ]:
roi = ROIS[0]
experiments = get_experiments_for_roi(roi)
experiments

## Define the data

Now we load the results from both experiments and concatenate it to produce a single dataframe. We also add a `"montage"` column to keep track of which type of montage the records were acquired with (`"bipolar"` or `"unipolar"`).

In [ ]:
dfs = list()
for e in experiments:
    df = e.get_data()
    df["montage"] = "bipolar" if e.is_bipolar else "unipolar"
    dfs.append(df[["sub", "k", "montage", "e", "e_r"]])
data = pd.concat(dfs, axis=0, ignore_index=True)

We can then display the results for the mangitude of the electric field $| \pmb{e} |$ and of its component normal to the cortical surface $| \pmb{e}_r |$.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
plot_bipolar_unipolar(axs[0], "e", data, LABELS["e"], CAPTIONS["e"])
plot_bipolar_unipolar(axs[1], "e_r", data, LABELS["e_r"], CAPTIONS["e_r"])
plt.show()
columns = ["mean", "std", "25%", "75%"]
display_side_by_side(
    [
        data.groupby("montage").describe()["e"][columns],
        data.groupby("montage").describe()["e_r"][columns],
    ],
    ["", ""],
)

## Define the base model

We can now model the difference between the means of the two populations (T-test) using the following Bayesian models:
$$
| \pmb{e} | \sim \mathcal{N}(\mu_{| \pmb{e} |}, \sigma_{| \pmb{e} |}^2), \\
| \pmb{e}_r | \sim \mathcal{N}(\mu_{| \pmb{e}_r |}, \sigma_{| \pmb{e}_r |}^2),
$$
where
$$
\mu_{| \pmb{e} |} = \alpha_{| \pmb{e} |} + \beta_{| \pmb{e} |} \cdot X_\text{uni} + \varepsilon_{| \pmb{e} |}, \\
\mu_{| \pmb{e}_r |} = \alpha_{| \pmb{e}_r |} + \beta_{| \pmb{e}_r |} \cdot X_\text{uni} + \varepsilon_{| \pmb{e}_r |}.
$$

The values of $X_\text{uni}$ are either $0$ if the montage is bipolar or $1$ if it is unipolar. This way, $\beta$ actually characterises the difference between the two experiments, meaning that if $\beta \approx 0$ the two populations are similar.

## Pooled model

The first models we fit are fully pooled. This means that we do not account for any hierarchy in the data and instead consider all the samples as independent.

These models are exactly the ones described above. 

### Fit the models

Weakly informative priors are set automatically for $\alpha$, $\beta$ and $\sigma$.

In [ ]:
model_pooled_e = bmb.Model("e ~ montage", data)
model_pooled_e_r = bmb.Model("e_r ~ montage", data)

In [ ]:
trace_pooled_e = model_pooled_e.fit(draws=1000, chains=4, random_seed=RANDOM_SEED)
trace_pooled_e_r = model_pooled_e_r.fit(draws=1000, chains=4, random_seed=RANDOM_SEED)

### Results

We display the probability density of $\beta$ for both models as well as their $95$% HDI.

In [ ]:
idx = ["alpha", "beta", "sigma"]
summary_pooled_e = az.summary(trace_pooled_e, stat_funcs=FUNC_DICT, extend=False)
summary_pooled_e_r = az.summary(trace_pooled_e_r, stat_funcs=FUNC_DICT, extend=False)
summary_pooled_e.index, summary_pooled_e_r.index = idx, idx

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 2.5))
plot_posterior(
    axs[0],
    trace_pooled_e,
    summary_pooled_e,
    "montage",
    LABELS["e"],
    CAPTIONS["e"],
    summary_param="beta",
    show_zero=True,
)
plot_posterior(
    axs[1],
    trace_pooled_e_r,
    summary_pooled_e_r,
    "montage",
    LABELS["e_r"],
    CAPTIONS["e_r"],
    summary_param="beta",
    show_zero=True,
)
fig.tight_layout()
plt.show()
display_side_by_side([summary_pooled_e, summary_pooled_e_r], ["", ""])

## Hierarchical model

Now, we consider the fact that multiple records were obtained with the same subject.

To account for this, the definitions of the intercept $\alpha$ and the slope $\beta$ become
$$
\alpha_i = \alpha_\text{common} + \alpha_{\text{sub}_i}, \\
\beta_i = \beta_\text{common} + \beta_{\text{sub}_i}.
$$

Here, we differenciate the common value of the intercept $\alpha_\text{common}$ and of the slope $\beta_\text{common}$ from the subject specific contributions $\alpha_{\text{sub}_i}$ and $\beta_{\text{sub}_i}$. These contributions are modelled as centered normal distributions
$$
\alpha_\text{sub} \sim \mathcal{N}(0, \sigma_{\alpha_\text{sub}}), \\
\beta_\text{sub} \sim \mathcal{N}(0, \sigma_{\beta_\text{sub}}).
$$

Hence, the models become
$$
| \pmb{e} | \sim \mathcal{N}(\mu_{| \pmb{e} |, i}, \sigma_{| \pmb{e} |}^2), \\
| \pmb{e}_r | \sim \mathcal{N}(\mu_{| \pmb{e}_r |, i}, \sigma_{| \pmb{e}_r |}^2),
$$
where
$$
\mu_{| \pmb{e} |, i} = \alpha_{| \pmb{e} |, i} + \beta_{| \pmb{e} |, i} \cdot X_\text{uni} + \varepsilon_{| \pmb{e} |}, \\
\mu_{| \pmb{e}_r |, i} = \alpha_{| \pmb{e}_r |, i} + \beta_{| \pmb{e}_r |, i} \cdot X_\text{uni} + \varepsilon_{| \pmb{e}_r |}.
$$

### Fit the models

Once again, weakly informative priors are automatically set for all the parameters.

In [ ]:
model_hierarchic_e = bmb.Model("e ~ montage + (montage | C(sub))", data)
model_hierarchic_e_r = bmb.Model("e_r ~ montage + (montage | C(sub))", data)

In [ ]:
trace_hierarchic_e = model_hierarchic_e.fit(
    draws=1000, chains=4, random_seed=RANDOM_SEED, target_accept=0.9
)
trace_hierarchic_e_r = model_hierarchic_e_r.fit(
    draws=1000, chains=4, random_seed=RANDOM_SEED, target_accept=0.9
)

### Results

Again, we display the posteriors of $\beta$ and their $95$% HDI for the two metrics.

In [ ]:
idx = [
    "alpha",
    "beta",
    "sigma(alpha_sub)",
    *[f"alpha_sub_{i}" for i in range(len(data["sub"].unique()))],
    "sigma(beta_sub)",
    *[f"beta_sub_{i}" for i in range(len(data["sub"].unique()))],
    "sigma",
]
summary_hierarchic_e = az.summary(
    trace_hierarchic_e, stat_funcs=FUNC_DICT, extend=False
)
summary_hierarchic_e_r = az.summary(
    trace_hierarchic_e_r, stat_funcs=FUNC_DICT, extend=False
)
summary_hierarchic_e.index, summary_hierarchic_e_r.index = idx, idx

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 2.5))
plot_posterior(
    axs[0],
    trace_hierarchic_e,
    summary_hierarchic_e,
    "montage",
    LABELS["e"],
    CAPTIONS["e"],
    summary_param="beta",
    show_zero=True,
)
plot_posterior(
    axs[1],
    trace_hierarchic_e_r,
    summary_hierarchic_e_r,
    "montage",
    LABELS["e_r"],
    CAPTIONS["e_r"],
    summary_param="beta",
    show_zero=True,
)
fig.tight_layout()
plt.show()
rows = ["alpha", "sigma(alpha_sub)", "beta", "sigma(beta_sub)", "sigma"]
display_side_by_side(
    [summary_hierarchic_e.loc[rows, :], summary_hierarchic_e_r.loc[rows, :]], ["", ""]
)